In [ ]:
# default_exp inferno

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# INFERNO loss

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
from pytorch_inferno.model_wrapper import ModelWrapper
from pytorch_inferno.callback import *
from pytorch_inferno.data import get_paper_data
from pytorch_inferno.plotting import *
from pytorch_inferno.inference import *
from pytorch_inferno.utils import *

from fastcore.all import partialler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from typing import *
from collections import OrderedDict
from fastcore.all import store_attr

import torch.nn as nn
from torch import optim
import torch
from torch import Tensor
from torch import autograd

# Network

In [ ]:
bs = 200
data, test = get_paper_data(200000, bm=0, bs=bs, n_test=1000000)

In [ ]:
# export
class VariableSoftmax(nn.Softmax):
    def __init__(self, temp:float=1, dim:int=-1):
        super().__init__(dim=dim)
        self.temp = temp
        
    def forward(self, x:Tensor) -> Tensor: return super().forward(x/self.temp)

In [ ]:
x = torch.randn((1,10))

In [ ]:
VariableSoftmax(0.1)(x), VariableSoftmax(0.5)(x), VariableSoftmax(1)(x)

(tensor([[2.0492e-08, 9.9981e-01, 1.8703e-10, 1.8962e-15, 3.3516e-14, 1.8583e-04,
          1.3037e-16, 1.8234e-10, 9.0611e-06, 5.0900e-15]]),
 tensor([[2.1731e-02, 7.4947e-01, 8.4948e-03, 8.5182e-04, 1.5129e-03, 1.3446e-01,
          4.9866e-04, 8.4518e-03, 7.3487e-02, 1.0378e-03]]),
 tensor([[0.0753, 0.4422, 0.0471, 0.0149, 0.0199, 0.1873, 0.0114, 0.0470, 0.1385,
          0.0165]]))

In [ ]:
net = nn.Sequential(nn.Linear(3,100),  nn.ReLU(),
                    nn.Linear(100,100),nn.ReLU(),
                    nn.Linear(100,10), VariableSoftmax(0.1))
init_net(net)

# Loss

In [ ]:
x,y,w = next(iter(data.trn_dl))
preds = net(x)
assert preds.shape == (bs,10)